In [13]:
import numpy as np
import matplotlib.pyplot as plt

eps = 0.01

d = 0.001


def grad_desc(func, grad, lr, x, eps):
    points = np.array(x)

    while True:
        prev_x = x
        x = x - lr(x) * np.array(grad(x[0], x[1]))
        points = np.concatenate((points, x))
        if abs(func(x[0], x[1]) - func(prev_x[0], prev_x[1])) < eps:
            break

    return points


def grad_func(x, y):
    return np.array((2 * (x - 4), -2 * (5 - y)))


def func(x, y):
    return np.array((x - 4) ** 2 + (5 - y) ** 2)


def dichotomy(x, a_1, a_2):
    points = np.array(x)
    while True:
        if abs(a_1 - a_2) < eps: break
        new_a_1 = (a_1 + a_2) / 2 - d
        new_a_2 = (a_1 + a_2) / 2 + d
        new_a_1_ = func((x - new_a_1 * np.array(grad_func(x[0], x[1])))[0],
                        (x - new_a_1 * np.array(grad_func(x[0], x[1])))[1])
        new_a_2_ = func((x - new_a_2 * np.array(grad_func(x[0], x[1])))[0],
                        (x - new_a_2 * np.array(grad_func(x[0], x[1])))[1])
        points = np.concatenate((points, x - new_a_1 * np.array(grad_func(x[0], x[1]))))
        points = np.concatenate((points, x - new_a_2 * np.array(grad_func(x[0], x[1]))))
        if new_a_2_ > new_a_1_:
            a_2 = new_a_2
        else:
            a_1 = new_a_1
    return points


dichotomy(np.array((20, 20)), -10, 10)

array([ 2.00000000e+01,  2.00000000e+01,  2.00320000e+01,  2.00300000e+01,
        1.99680000e+01,  1.99700000e+01, -1.39952000e+02, -1.29955000e+02,
       -1.40016000e+02, -1.30015000e+02, -5.99600000e+01, -5.49625000e+01,
       -6.00240000e+01, -5.50225000e+01, -1.99640000e+01, -1.74662500e+01,
       -2.00280000e+01, -1.75262500e+01,  3.40000000e-02,  1.28187500e+00,
       -3.00000000e-02,  1.22187500e+00,  1.00330000e+01,  1.06559375e+01,
        9.96900000e+00,  1.05959375e+01,  5.03350000e+00,  5.96890625e+00,
        4.96950000e+00,  5.90890625e+00,  2.53375000e+00,  3.62539063e+00,
        2.46975000e+00,  3.56539063e+00,  3.78362500e+00,  4.79714844e+00,
        3.71962500e+00,  4.73714844e+00,  4.40856250e+00,  5.38302734e+00,
        4.34456250e+00,  5.32302734e+00,  4.09609375e+00,  5.09008789e+00,
        4.03209375e+00,  5.03008789e+00,  3.93985938e+00,  4.94361816e+00,
        3.87585938e+00,  4.88361816e+00])